In [44]:
import pandas as pd
import numpy as np
import pickle
import os

def prepare_input_data(input_dict):
    """
    Prepare input data to match the model's expected features.
    
    Parameters:
    -----------
    input_dict : dict
        Dictionary containing input values for prediction
        Must contain basic process parameters and material groups
    
    Returns:
    -------
    pd.DataFrame
        DataFrame with all required features in correct order
    """
    # Create a template with all possible material combinations set to 0
    material_columns = {
        'Powder_Material_Group_Cobalt_based': 0,
        'Powder_Material_Group_Copper_based': 0,
        'Powder_Material_Group_Nickel_based': 0,
        'Powder_Material_Group_Steel_based': 0,
        'Powder_Material_Group_Titanium_based': 0,
        'Substrate_Material_Group_Nickel_based': 0,
        'Substrate_Material_Group_Steel_based': 0,
        'Substrate_Material_Group_Titanium_based': 0
    }
    
    # Set the appropriate material columns to 1 based on input
    if 'powder_material' in input_dict:
        powder_col = f"Powder_Material_Group_{input_dict['powder_material']}"
        if powder_col in material_columns:
            material_columns[powder_col] = 1
            
    if 'substrate_material' in input_dict:
        substrate_col = f"Substrate_Material_Group_{input_dict['substrate_material']}"
        if substrate_col in material_columns:
            material_columns[substrate_col] = 1

    
    # Create DataFrame with all required features
    input_data = {
        'Power (W)': input_dict.get('power', 0),
        'Mass Flowrate (g/min)': input_dict.get('mass_flowrate', 0),
        'Travel Velocity (mm/min)': input_dict.get('travel_velocity', 0),
        'Height (mm)': input_dict.get('height', 0),
        'Contact Angle (deg)': input_dict.get('contact_angle', 0),
        'Size_Width_Ratio': input_dict.get('Spot Size (mm)', 0) / input_dict.get('Width (mm)', 1)
    }
    
    # Combine process parameters with material columns
    input_data.update(material_columns)
    
    # Create DataFrame and ensure correct column order
    expected_columns = ['Power (W)', 'Mass Flowrate (g/min)', 'Travel Velocity (mm/min)', 
                       'Height (mm)', 'Contact Angle (deg)', 'Size_Width_Ratio',
                       'Powder_Material_Group_Cobalt_based', 'Powder_Material_Group_Copper_based',
                       'Powder_Material_Group_Nickel_based', 'Powder_Material_Group_Steel_based',
                       'Powder_Material_Group_Titanium_based', 'Substrate_Material_Group_Nickel_based',
                       'Substrate_Material_Group_Steel_based', 'Substrate_Material_Group_Titanium_based']
    
    df = pd.DataFrame([input_data])[expected_columns]
    return df



In [45]:
def predict_aspect_ratio(input_dict, model_name='random_forest'):
    """
    Make predictions using the saved model.
    
    Parameters:
    -----------
    input_dict : dict
        Dictionary containing input parameters
    model_name : str
        Name of the model to use ('linear_regression', 'random_forest', or 'xgboost')
    
    Returns:
    --------
    float
        Predicted aspect ratio
    """
    # Load the model
    model_path = os.path.join('./saved_models', f'{model_name}_best_model.pkl')
    with open(model_path, 'rb') as file:
        model = pickle.load(file)
    
    # Prepare input data
    input_df = prepare_input_data(input_dict)
    
    # Make prediction
    prediction = model.predict(input_df)[0]
    
    return prediction

In [46]:
# Example usage:
input_parameters = {
    'power': 0.839125,  # Power in Watts
    'mass_flowrate': 0.759631,  # g/min
    'travel_velocity': 7.538071,  # mm/min
    'height': 0.265605,  # mm
    'contact_angle': 108.400000,  # degrees
    'Spot Size (mm)':0,
    'Width (mm)': 0.1,
    'powder_material': 'Steel_based',
    'substrate_material': 'Steel_based'
}

# Make predictions using different models
for model_name in ['linear_regression', 'random_forest', 'xgboost']:
    try:
        prediction = predict_aspect_ratio(input_parameters, model_name)
        print(f"\n{model_name.title()} prediction:")
        print(f"Predicted Aspect Ratio: {prediction:.4f}")
    except Exception as e:
        print(f"Error with {model_name}: {str(e)}")


Linear_Regression prediction:
Predicted Aspect Ratio: 0.6461

Random_Forest prediction:
Predicted Aspect Ratio: 0.6306

Xgboost prediction:
Predicted Aspect Ratio: 0.6432
